<a href="https://colab.research.google.com/github/jeha100/google_drive/blob/main/temp/chapter_5_2_%ED%95%98%EC%A7%80%EC%9D%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
- 검증 세트는 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어낸 데이터 세트이다
- 교차 검증은 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련한다.
- 그리드 서치는 하이퍼파라미터 탐색을 자동화해 주는 도구
- 랜덤 서치는 연속된 매개변수 값 탐색시에 유용하다. 

# 교차 검증과 그리드 서치
- 키워드 : 하이퍼 파라미터 (그리드서치 vs 랜덤서치)
- 데이터가 작을 때, 주로 사용 
- 하이퍼파라미터
  + max_depth: 3 , 정확도가 84% 
- 결론 
  + 모르면 디폴트만 쓰자!
  + 가성비(시간 대비 성능 보장 안됨!)


# 검증 세트
- 테스트 세트 (1회성)
- 훈련 데이터를 훈련 데이터 + 검증 데이터로 재 분할 

## 현실
- 테스트 데이터가 별도로 존재하지 않음! 
- 전체 데이터 = 훈련(6) : 검증(2) : 테스트(2) 
  + 테스트 데이터는 모르는 데이터로 생각 ! 

In [3]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [8]:
print(sub_input.shape, val_input.shape,test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


# 모델 만든 후 평가 
- 과대 적합 발생

In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증
- 교차 검증의 목적 : 좋은 모델이 만들어진다! 
  + 좋은 모델 != 성능 좋은 모델
  + 좋은 모델 = 과대적합이 아닌 모델 = 모형의 오차가 적은 모델 = 안정적인 모델
- 교재 245p
  + 모델평가 1 : 90% (소요시간 : 1시간)
  + 모델평가 2 : 85% 
  + 모델평가 3 : 80% 
- 단점 : 시간이 오래 걸림 

# 교차 검증 함수

In [9]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01549745, 0.01021481, 0.01882243, 0.022264  , 0.0100112 ]), 'score_time': array([0.00123429, 0.00103593, 0.00128913, 0.00123119, 0.00113535]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 최종점수 평균 구하기

In [10]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


- 훈련 세트 섞은 후 , 10- 폴드 교차검증

In [11]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
# 3가지 방법 중 가장 많이 쓰이는 함수 
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼 파라미터 튜닝 꼭 하고 싶다! 
- 랜덤 서치 사용하자! 
- 자동으로 잡아주는 라이브러리들이 등장하기 시작함
  + hyperopt

In [19]:
%%time

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # 딕셔너리 

# dt = DecisionTreeClassifier(random_state = 42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) 
gs.fit(train_input, train_target)# 모형 실행 타이밍 -시간 걸리기 시작 마차 두마리? 세마리?

CPU times: user 68.6 ms, sys: 3.55 ms, total: 72.2 ms
Wall time: 170 ms


In [21]:
dt = gs.best_estimator_
print(dt)
print(dt.score(train_input, train_target))# 성능 
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

# 랜덤 서치
- p.252, 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있도록 확률 분포 객체를 전달. 

In [24]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([5, 0, 2, 1, 0, 2, 6, 6, 6, 4])

In [25]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 83, 114,  95,  97, 111,  84,  89, 118, 102, 107]))

In [27]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [28]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f23f55f0e90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f23f55f0fd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f23f5682990>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f23f55f0cd0>},
                   random_state=42)